In [1]:
import tabula
import os
import glob
import pandas
from sqlalchemy import create_engine
import pymysql
import pyodbc
from datetime import datetime   

import database

In [2]:
USER = database.USER
PASSWORD = database.PASSWORD
SERVER = database.SERVER
DATABASE = database.DATABASE
TBL = database.TABLE

## In this hypothetical scenario we have a new daily pdf, we are sure that the table always has this structure.
### 

![alt text](PDFspreadsheet.png "Title")

# Extract pdf

In [3]:
def readfiles(path):
    pdfs = []
    for file in glob.glob(f"{path}*.pdf"):
        pdfs.append(file)
    return pdfs

In [4]:
def extracttables(pdfs):
    for pdf in pdfs:
        tables = tabula.read_pdf(pdf, pages="all")
    return tables


In [5]:
def extract(path):
    pdfs = readfiles(path)
    tables = extracttables(pdfs)
    return tables
    

# Transform

In [6]:
def renamecolumns(tables):
    for table in tables:
        old_cols = list(table.columns)
        new_col0 = ''.join(table.iloc[[0,1],0].values).lower()
        new_col1 = ''.join(table.iloc[[0,1],1].values).split('(')[0].lower()
        new_col2 = ''.join(table.iloc[[0,1],2].values).split('(')[0].lower()
        new_col3 = ''.join(table.iloc[[1,2],3].values).lower()
        new_col5 = ''.join(table.iloc[[1,2],5].values).lower()
        table.rename(columns={old_cols[0] : new_col0,
                           old_cols[1] : new_col1,
                           old_cols[2] : new_col2,
                           old_cols[3] : new_col3,
                           old_cols[5] : new_col5},inplace = True)

    return tables

In [7]:
def splitcolumn(tables):
    new_tables = []
    for table in tables:
        new_cols = table['Percent Fuel Savings'].str.split(' ', expand=True)
        old_cols = list(new_cols.columns)
        new_col0 = ''.join(new_cols.iloc[[1,2],0].values).lower()
        new_col1 = ''.join(new_cols.iloc[[1,2],1].values).lower()
        new_cols.rename(columns={old_cols[0] : new_col0,
                       old_cols[1] : new_col1},inplace = True)
        table.drop(table.columns[[4]],axis = 1,inplace = True)
        table = table.join(new_cols)
        table = table[['cyclename', 'ki', 'distance', 'improvedspeed', 'decreasedaccel',
                                   'eliminatestops','decreasedidle']]
        new_tables.append(table)

    return new_tables

In [8]:
def removerows(tables):
    new_tables = []
    for table in tables:
        table = table.iloc[3:,:]
        new_tables.append(table)
    return new_tables

In [9]:
def remocecharacter(tables):
    new_tables = []
    for table in tables:
        for col in table.columns[3:]:
            table[col] = table[col].map(lambda x: str(x).replace('%',''))
        new_tables.append(table)
    return tables

In [10]:
def addDate(tables):
    new_tables = []
    for table in tables:
        today = datetime.now()
        today.strftime('%Y-%m-%d %H:%M:%S')
        table['date'] = today
        new_tables.append(table)
    return new_tables
    

In [11]:
def transform(tables):
    tables = renamecolumns(tables)
    tables = splitcolumn(tables)
    tables = removerows(tables)
    tables = remocecharacter(tables)
    tables = addDate(tables)
    
    return tables
    

# Load

In [12]:
def load(tables):
    try:
        rows_imported = 0
        engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{SERVER}/{DATABASE}")
        for table in tables:
            print(f'Importing rows {rows_imported} to {rows_imported + len(table)} ... for table {TBL}')
            # Save df to mysql
            table.to_sql(TBL, engine, if_exists = 'append', index = False)
            rows_imported +- len(table)
            # add elapsed time to final print out
            print("Data imported seccesful")
    except Exception as e:
        print('Data load error: '+ str(e))

In [13]:
if __name__ == "__main__":
    path = 'FuelData/'
    tables = extract(path)
    tables = transform(tables)
    load(tables)



Importing rows 0 to 5 ... for table fuel
Data imported seccesful


![alt text](db-result.png "Result")